In [1]:
pip install transformers

In [2]:
# import nltk.data
# nltk.download('punkt')

# # Define the input and output file paths
# input_file_path = "/content/webpage_data.txt"  # Replace with your input file path
# output_file_path = "/content/tokenized_webpage_data.txt"  # Replace with your output file path

# # Initialize the sentence tokenizer
# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# # Read the input text from the file
# with open(input_file_path, 'r', encoding='utf-8') as input_file:
#     input_text = input_file.read()

# # Tokenize the input text into sentences
# sentences = tokenizer.tokenize(input_text)

# # Write each sentence to the output file on a new line
# with open(output_file_path, 'w', encoding='utf-8') as output_file:
#     for sentence in sentences:
#         output_file.write(sentence.strip() + '\n')

# # import re
# # re.sub(r'[^a-zA-Z0-9\s]', '', output_file_path) #Removing special characters

# print(f"Sentences tokenized and saved to '{output_file_path}'.")


In [3]:
!pip install transformers[torch,accelerate]>=0.20.1



In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load the pretrained GPT-2 model and tokenizer
model_name = "gpt2"  # You can choose a different model if needed
model = GPT2LMHeadModel.from_pretrained(model_name).to('cuda:0')
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# from transformers import RobertaForCausalLM, RobertaTokenizer

# # Load the RoBERTa model and tokenizer
# model_name = "roberta"
# model = RobertaForCausalLM.from_pretrained("roberta").to('cuda:0')  # Move to GPU
# tokenizer = RobertaTokenizer.from_pretrained("roberta")


# Collect and preprocess your data (replace with actual data collection and preprocessing)
data = "/content/Autograder_style.txt"#"/content/tokenized_webpage_data.txt"

# Tokenize and format the data for training
train_dataset = TextDataset(tokenizer=tokenizer, file_path=data, block_size=128)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine-tuned-model",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# Create a Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model()


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


In [6]:
task=input("Enter task: ")
code=input("Enter code: ")
report=input("Enter report: ")
user_prompt = f" Evaluate the {code} and {report} based on how well they satisfy the {task}. Assign a score(out of 50) and provide a detailed feedback"


ft_model_name="./fine-tuned-model"
ft_model = GPT2LMHeadModel.from_pretrained(ft_model_name).to('cuda:0')
# tokenizer = GPT2Tokenizer.from_pretrained(ft_model_name)


response =model.generate(
    tokenizer.encode(user_prompt, return_tensors="pt").to('cuda:0'),
    max_length=50,  # Adjust the maximum response length as needed
    num_return_sequences=1,
    no_repeat_ngram_size=2,
)

autograder = tokenizer.decode(response[0], skip_special_tokens=True)
print("Autograder:", autograder)


Enter task: You will create 3 required functions for MLPs (you may create more functions for your own use, but you need to at least create these two as specified): train_MLP_model_average(path_to_train_file) This method trains a multi-layer perceptron model on some training data and returns that trained model. The training texts should be represented by the average of word2vec or glove embeddings. You may use any pretrained word2vec or glove embeddings you choose. Your MLP should be 3 layers with 100 neurons per layer. The format for the train file should follow the same format as the training data file! train_MLP_model_student(path_to_train_file): This method is the same as above, however, you need to choose an alternate way to represent the input text with embeddings that is not the average. The MLP slides had some possible solutions, so you may choose any of these or your own (but you should always note your decisions). You may also try out multiple but make sure you note which work

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Autograder:  Evaluate the def train_MLP_model_student(path_to_train_file):     # Load the IMDB dataset     X, y = load_imdb_dataset(path_to_train_file)      # Encode labels (positive/negative) to numerical values (0/1)     label_encoder = LabelEncoder()     y = label_encoder.fit_transform(y)      # Load pre-trained Word2Vec embeddings (You need to download this)     word2vec_model = model      # Preprocess the text and compute average embeddings     def sum_word_embeddings(text):         words = text.split()         embeddings = [word2vec_model[word] for word in words if word in word2vec_model]         if len(embeddings) > 0:             return np.sum(embeddings, axis=0) #Here we just add the embeddings instead of averaging them         else:             return np.zeros(word2vec_model.vector_size)      X_sum = np.array([sum_word_embeddings(text) for text in X])      # Create and train the MLP model     mlp_model = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000, random_

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 895, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
